In [131]:
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
import cv2 as cv

In [132]:
def in_Range(hsv, lower_range, upper_range):
    height = hsv.shape[0]
    width = hsv.shape[1]
    channel = hsv.shape[2]
    l1, l2, l3 = lower_range
    u1, u2, u3 = upper_range
    mask = hsv.copy()
    for i in range(height):
        for j in range(width):
            if hsv[i][j][0] >= l1 and hsv[i][j][1] >= l2 and hsv[i][j][2] >= l3 and hsv[i][j][0] <= u1 and hsv[i][j][1] <= u2 and hsv[i][j][2] <= u3:
                    mask[i][j][:] = 0
    return mask

In [150]:
def invert_channel(img):
    bgr = img.copy()
    # to convert from RGB to BGR
    for i in range(img.shape[0]):
        for j in range(img.shape[1]):
            bgr[i][j][2] = img[i][j][0]
            bgr[i][j][1] = img[i][j][1]
            bgr[i][j][0] = img[i][j][2]
    return bgr

In [134]:
def threshold(img, thresh, value):   
    new = np.empty((img.shape[0], img.shape[1], 3), dtype=np.uint8)
    
    for x in range(img.shape[0]):
        for y in range(img.shape[1]):
            if img[x,y] > thresh:
                new[x,y, :] = value
            else:
                new[x,y, :] = 0
    return new

In [135]:
img1 = np.array(Image.open('roi.jpg'))
img1 = invert_channel(img1)
img2 = np.array(Image.open('ball.jpg'))
img2 = invert_channel(img2)
# Region in img1 where i want to put the ball image
r, w, c = img2.shape
roi_start_x = 855
roi_start_y = 220
roi = img1[roi_start_x:roi_start_x+r, roi_start_y:roi_start_y+w, :]

mask_body = cv.cvtColor(img2,cv.COLOR_BGR2GRAY)
mask_body = threshold(mask_body, 165, 255)

mask_inv_body = np.bitwise_not(mask_body)

# Now black-out the area of logo in ROI
img1_bg = np.bitwise_and(roi, mask_inv_body)

# Take only region of logo from logo image.
img2_fg = np.bitwise_and(img2, mask_body)

In [136]:
mask_img = Image.fromarray(mask_body)
mask_img.show()

In [137]:
front_img = Image.fromarray(np.uint8(img2_fg))
front_img.show()

In [138]:
bg_img = Image.fromarray(np.uint8(img1_bg))
bg_img.show()

In [148]:
# Put logo in ROI and modify the main image
# dst = cv.add(img1_bg,img2_fg)
dst = np.bitwise_or(img1_bg, img2_fg)
dst_img = Image.fromarray(np.uint8(dst))
dst_img.show()

In [154]:
img1[roi_start_x:roi_start_x+r, roi_start_y:roi_start_y+w, :] = dst
final = invert_channel(img1)
final = Image.fromarray(np.uint8(final))
final.show()